# 使用鸢尾花数据集构建决策树分类器

## 1. 先使用sklearn对数据进行处理（获取、拆分）

### 使用sklearn 获取自带的小数据集

In [4]:
from sklearn.model_selection import train_test_split # 训练集测试集拆分
from sklearn import svm   # 引入支持向量机
from sklearn import tree  # 引入决策树

iris_data = load_iris()  # 导入数据
print('sklearn 提供的数据集格式为：', type(iris_data))
print('sklearn 提供的数据集中特征与标签格式为：', type(iris_data.data), type(iris_data.target))
print('鸢尾花数据集特征集的特征向量维度：', iris_data.data.shape)
print('鸢尾花数据集特征集的标签维度：', iris_data.target.shape)
print('鸢尾花数据集特征集的标签含义：', iris_data.target_names)

sklearn 提供的数据集格式为： <class 'sklearn.utils.Bunch'>
sklearn 提供的数据集中特征与标签格式为： <class 'numpy.ndarray'> <class 'numpy.ndarray'>
鸢尾花数据集特征集的特征向量维度： (150, 4)
鸢尾花数据集特征集的标签维度： (150,)
鸢尾花数据集特征集的标签含义： ['setosa' 'versicolor' 'virginica']


### 使用sklearn 获取的鸢尾花数据集部分示例

In [5]:
iris_features = iris_data.data
iris_labels = iris_data.target

print('鸢尾花数据集特征集的特征向量前十条为：\n', iris_features[:10])
print('鸢尾花数据集特征集的标签前十条为：\n', iris_labels[:10])

鸢尾花数据集特征集的特征向量前十条为：
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
鸢尾花数据集特征集的标签前十条为：
 [0 0 0 0 0 0 0 0 0 0]


### 拆分数据集（测试数据集占30%，设置随机种子为21）

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(iris_features, iris_labels, test_size=0.3, random_state=21)

## 2.1 SVM模型训练

### 参数说明：
常用的SVM参数包括C、kernel、gamma，其他参数可基于应用场景进行调整，详细参数如下：

|名称|描述|
|:-|:-| 
|C|惩罚系数，用来控制损失函数的惩罚系数，类似于LR中的正则化系数。C越大，相当于惩罚松弛变量，希望松弛变量接近0，即对误分类的惩罚增大，趋向于对训练集全分对的情况，这样会出现训练集测试时准确率很高，但泛化能力弱，容易导致过拟合。 C值小，对误分类的惩罚减小，容错能力增强，泛化能力较强，但也可能欠拟合。|
|kernel|算法中采用的核函数类型，核函数是用来将非线性问题转化为线性问题的一种方法。参数选择有RBF, Linear, Poly, Sigmoid，precomputed或者自定义一个核函数, 默认的是"RBF"，即径向基核，也就是高斯核函数；而Linear指的是线性核函数，Poly指的是多项式核，Sigmoid指的是双曲正切函数tanh核。|
|degree|当指定kernel为'poly'时，表示选择的多项式的最高次数，默认为三次多项式；若指定kernel不是'poly'，则忽略，即该参数只对'poly'有用。（多项式核函数是将低维的输入空间映射到高维的特征空间）|
|gamma|核函数系数，该参数是rbf，poly和sigmoid的内核系数；默认是'auto'，那么将会使用特征位数的倒数，即1 / n_features。（即核函数的带宽，超圆的半径）。gamma越大，σ越小，使得高斯分布又高又瘦，造成模型只能作用于支持向量附近，可能导致过拟合；反之，gamma越小，σ越大，高斯分布会过于平滑，在训练集上分类效果不佳，可能导致欠拟合。|
|coef0|核函数常数值(y=kx+b中的b值)， 只有‘poly’和‘sigmoid’核函数有，默认值是0。|
|shrinking|是否进行启发式。如果能预知哪些变量对应着支持向量，则只要在这些样本上训练就够了，其他样本可不予考虑，这不影响训练结果，但降低了问题的规模并有助于迅速求解。进一步，如果能预知哪些变量在边界上(即a=C)，则这些变量可保持不动，只对其他变量进行优化，从而使问题的规模更小，训练时间大大降低。这就是Shrinking技术。 Shrinking技术基于这样一个事实：支持向量只占训练样本的少部分，并且大多数支持向量的拉格朗日乘子等于C。|
|probability|是否使用概率估计，默认是False。 必须在 fit( ) 方法前使用，该方法的使用会降低运算速度。|
|tol|残差收敛条件，默认是0.0001，即容忍1000分类里出现一个错误， 与LR中的一致；误差项达到指定值时则停止训练。|
|cache_size|缓冲大小，用来限制计算量大小，默认是200M。|
|class_weight|字典类型或者'balance'字符串。权重设置，正类和反类的样本数量是不一样的，这里就会出现类别不平衡问题，该参数就是指每个类所占据的权重，默认为1，即默认正类样本数量和反类一样多，也可以用一个字典dict指定每个类的权值，或者选择默认的参数balanced，指按照每个类中样本数量的比例自动分配权值。如果不设置，则默认所有类权重值相同，以字典形式传入。 将类i 的参数C设置为SVC的class_weight[i]*C。如果没有给出，所有类的weight 为1。'balanced'模式使用y 值自动调整权重，调整方式是与输入数据中类频率成反比。如n_samples / (n_classes * np.bincount(y))。（给每个类别分别设置不同的惩罚参数C，如果没有给，则会给所有类别都给C=1，即前面参数指出的参数C。如果给定参数'balance'，则使用y的值自动调整与输入数据中的类频率成反比的权重。）|
|verbose|是否启用详细输出。在训练数据完成之后，会把训练的详细信息全部输出打印出来，可以看到训练了多少步，训练的目标值是多少；但是在多线程环境下，由于多个线程会导致线程变量通信有困难，因此verbose选项的值就是出错，所以多线程下不要使用该参数。|
|max_iter|最大迭代次数，默认是-1，即没有限制。这个是硬限制，它的优先级要高于tol参数，不论训练的标准和精度达到要求没有，都要停止训练。|
|decision_function_shape|原始的SVM只适用于二分类问题，如果要将其扩展到多类分类，就要采取一定的融合策略，这里提供了三种选择。‘ovo’ 一对一，为one v one，即将类别两两之间进行划分，用二分类的方法模拟多分类的结果，决策所使用的返回的是（样本数，类别数*(类别数-1)/2）； ‘ovr’ 一对多，为one v rest，即一个类别与其他类别进行划分，返回的是(样本数，类别数)，或者None，就是不采用任何融合策略。默认是ovr，因为此种效果要比oro略好一点。|
|random_state|在使用SVM训练数据时，要先将训练数据打乱顺序，用来提高分类精度，这里就用到了伪随机序列。如果该参数给定的是一个整数，则该整数就是伪随机序列的种子值；如果给定的就是一个随机实例，则采用给定的随机实例来进行打乱处理；如果啥都没给，则采用默认的 np.random实例来处理。|

In [7]:
# 训练模型（设置各个参数的值）
svm_classifier = svm.SVC(C = 1.0, kernel = 'rbf', decision_function_shape = 'ovr', gamma = 0.01)
svm_classifier.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### SVM 模型评估

In [9]:
print('SVM训练集得分为：', svm_classifier.score(X_train, Y_train))
print('SVM测试集得分为：', svm_classifier.score(X_test, Y_test))

SVM训练集得分为： 0.9714285714285714
SVM测试集得分为： 0.8888888888888888


## 2.3 决策树模型训练

### 参数说明
决策树的常用参数包括criterion, max_depth, min_samples_leaf，其他参数可以根据应用场景进行调整。详细参数如下：

|名称|功能|描述|
|:-|:-|:-| 
|criterion|特征选择标准|‘gini’ or ‘entropy’ (default=”gini”)，前者是基尼系数，后者是信息熵。两种算法差异不大对准确率无影响，信息熵运算效率低一点，因为它有对数运算.一般说使用默认的基尼系数”gini”就可以了，即CART算法。除非你更喜欢类似ID3, C4.5的最优特征选择方法。| 
|splitter|特征划分标准|‘best’ or ‘random’ (default=”best”) 前者在特征的所有划分点中找出最优的划分点。后者是随机的在部分划分点中找局部最优的划分点。 默认的”best”适合样本量不大的时候，而如果样本数据量非常大，此时决策树构建推荐”random” 。| 
|max_depth|决策树最大深度|int or None, optional (default=None) 一般来说，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，推荐限制这个最大深度，具体的取值取决于数据的分布。常用的可以取值10-100之间。常用来解决过拟合。| 
|min_impurity_decrease|节点划分最小不纯度|float, optional (default=0.) 这个值限制了决策树的增长，如果某节点的不纯度(基尼系数，信息增益，均方差，绝对差)小于这个阈值，则该节点不再生成子节点。 sklearn 0.19.1版本之前叫 min_impurity_split。| 
|min_samples_split|内部节点再划分所需最小样本数|int, float, optional (default=2) 如果是 int，则取传入值本身作为最小样本数； 如果是 float，则去 ceil(min_samples_split * 样本数量) 的值作为最小样本数，即向上取整。| 
|min_samples_leaf|叶子节点最少样本数|如果是 int，则取传入值本身作为最小样本数； 如果是 float，则去 ceil(min_samples_leaf * 样本数量) 的值作为最小样本数，即向上取整。 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝。| 
|max_leaf_nodes|最大叶子节点数|int or None, optional (default=None) 通过限制最大叶子节点数，可以防止过拟合，默认是”None”，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制，具体的值可以通过交叉验证得到。| 
|min_impurity_split|信息增益的阀值|决策树在创建分支时，信息增益必须大于这个阀值，否则不分裂| 
|min_weight_fraction_leaf|叶子节点最小的样本权重和|float, optional (default=0.) 这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝。 默认是0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。|
|class_weight|类别权重|dict, list of dicts, “balanced” or None, default=None 指定样本各类别的的权重，主要是为了防止训练集某些类别的样本过多，导致训练的决策树过于偏向这些类别。这里可以自己指定各个样本的权重，或者用“balanced”，如果使用“balanced”，则算法会自己计算权重，样本量少的类别所对应的样本权重会高。当然，如果你的样本类别分布没有明显的偏倚，则可以不管这个参数，选择默认的”None” 不适用于回归树 sklearn.tree.DecisionTreeRegressor| 

In [10]:
# 创建决策树对象，使用信息熵作为依据
decision_tree_classifier = tree.DecisionTreeClassifier(criterion = 'entropy')

# fit方法分类。feature为原始数据集中的iris.data，labels为原始数据集中的iris.target
decision_tree_classifier = decision_tree_classifier.fit(iris_features, iris_labels)

### 决策树模型评估

In [11]:
print('Decision_tree训练集得分为：', decision_tree_classifier.score(X_train, Y_train))
print('Decision_tree测试集得分为：', decision_tree_classifier.score(X_test, Y_test))

Decision_tree训练集得分为： 1.0
Decision_tree测试集得分为： 1.0
